In [10]:
import mysql.connector
import pandas as pd

In [11]:
dldb = mysql.connector.Connect(
    host = "server",
    port = "3306",
    user = "user",
    password = "****",
    database = "database",
    use_unicode=True
)

In [11]:
cursor = dldb.cursor()
create_table_query = """
CREATE TABLE IF NOT EXISTS auction_price_data (
        species VARCHAR(16),
        origin VARCHAR(16),
        size VARCHAR(16),
        packaging VARCHAR(16),
        quantity FLOAT,
        highest FLOAT,
        lowest FLOAT,
        average FLOAT,
        date DATE
    )
    """
cursor.execute(create_table_query)


In [12]:

import os
from datetime import datetime, timedelta

start_date = datetime.strptime('2022-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2022-12-31', '%Y-%m-%d')
delta = timedelta(days=1)
cursor = dldb.cursor()
csv_file_template = './{:04d}/{:02d}/{:04d}-{:02d}-{:02d}.csv'
current_date = start_date

while current_date <= end_date:
    csv_file_path = csv_file_template.format(current_date.year, current_date.month, current_date.year, current_date.month, current_date.day)
    if os.path.exists(csv_file_path):
        try :
            df = pd.read_csv(csv_file_path, header=None, thousands=',',encoding='utf-8')
            df.columns = ['species', 'origin', 'size', 'packaging', 'quantity', 'highest', 'lowest', 'average', 'date']
            for index, row in df.iterrows():
                insert_query = """
                INSERT INTO auction_price_data (species, origin, size, packaging, quantity, highest, lowest, average, date)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                cursor.execute(insert_query, tuple(row))
            dldb.commit()
        except :
            pass
    current_date += delta
cursor.close()
dldb.close()